# Build a model of a protein family starting from a single sequence


1. Retrieve homologous (similar) sequences
2. Build a MSA from retrieved sequences
3. Caclulate a PSSM or HMM from the MSA
4. Evaluate the model against a ground truth dataset


### Implementation
For each step some alternative implementations that can have an effect on the model accuracy

**Step 1**
- Blast against a database (SwissProt, UniProt, RefSeq, NR, ReferenceProteomes, ...)

**Step 2**
- Build the MSA from Blast alignments 
- Build MSA realigning Blast hits (ClustalO, T-Coffee, Muscle, ...)
- Edit the MSA to remove poorly conserved columns
- Remove sequence redundancy, similar sequences, to improve model generalization
- Split the MSA, e.g. when there are multiple domains

**Step 3**
- Build a HMM using the HMMER command line 
- Build a PSSM using the Blast command line
- Skip this step and use the MSA in step 4 with HMMSEARCH

**Step 4**
- Define a ground truth dataset (Pfam, CATH, keywords, manual curation, ...)
- Calculate preicison, recall, MCC, balanced accuracy





In [2]:
from Bio import SeqIO
from Bio.Blast import NCBIXML
import pandas as pd
from pathlib import Path

# Generate a MSA

Align P06621 with the [NCBI Blast webservice](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PROGRAM=blastp&PAGE_TYPE=BlastSearch&LINK_LOC=blasthome)

Use to alternative databases
-SwissProt
-RefSeq selected

Set "Max target sequences" to 5000
When finish click on "Download" and select XML format

In [9]:

# Parse the XML output
data = []

# WARNING: if the code below crashes, remove "CREATE VIEW" lines manually from the XML file

blast_input = 'data/P06621_sprot.xml'  # SwissProt database
blast_input = 'data/P06621_refseq_selected.xml'  # "RefSeq selected" database

with open(blast_input) as f:
    blast_records = NCBIXML.parse(f)

    # Iterate Psiblast rounds
    for blast_record in blast_records:
        
        # Iterate query alignments
        query_id = blast_record.query
        for i, alignment in enumerate(blast_record.alignments):
            subject_id = alignment.title
            
            for hsp in alignment.hsps:
                data.append((query_id,
                                subject_id,
                                blast_record.query_length,
                                hsp.query,
                                hsp.match,
                                hsp.sbjct,
                                hsp.query_start,
                                hsp.query_end,
                                hsp.sbjct_start,
                                hsp.sbjct_end,
                                hsp.identities,
                                hsp.positives,
                                hsp.gaps,
                                hsp.expect,
                                hsp.score))
                
df = pd.DataFrame(data, columns=["query_id", "subject_id", "query_len",
                              "query_seq", "match_seq", "subject_seq",
                              "query_start", "query_end", 
                              "subject_start", "subject_end", "identity", "positive", "gaps", "eval", "bit_score"])
df

,query_id,subject_id,query_len,query_seq,match_seq,subject_seq,query_start,query_end,subject_start,subject_end,identity,positive,gaps,eval,bit_score
0,sp|P06621|CBPG_PSES6 Carboxypeptidase G2 OS=Ps...,gi|1545301071|ref|WP_126472623.1| M20/M25/M40 ...,415,MRPSIHRTAIAAVLATAFVAGTALAQKRDNVLFQAATDEQPAVIKT...,MRPSIHRTAIAAVLATAF+AGTALAQKRDNVLFQAATDEQPAVIKT...,MRPSIHRTAIAAVLATAFMAGTALAQKRDNVLFQAATDEQPAVIKT...,1,415,1,415,410,414,0,0.000000e+00,2129.0
1,sp|P06621|CBPG_PSES6 Carboxypeptidase G2 OS=Ps...,gi|1543118946|ref|WP_126023768.1| M20/M25/M40 ...,415,MRPSIHRTAIAAVLATAFVAGTA-LAQKRDNVLFQAATDEQPAVIK...,MRPSIHRTA+AA+LA AFVA A AQKRDNVLFQAATDEQPAVIK...,MRPSIHRTALAALLAAAFVAPAATWAQKRDNVLFQAATDEQPAVIK...,1,415,1,416,407,411,1,0.000000e+00,2097.0
2,sp|P06621|CBPG_PSES6 Carboxypeptidase G2 OS=Ps...,gi|1797811451|ref|WP_159279459.1| M20/M25/M40 ...,415,MRPSIHRTAIAAVLATAFVAGTALAQKRDNVLFQAATDEQPAVIKT...,MRPSIHRTA+AAVLATAFVAG A AQKRDNVLFQAATDEQPAVIKT...,MRPSIHRTAVAAVLATAFVAGGAWAQKRDNVLFQAATDEQPAVIKT...,1,415,1,415,393,408,0,0.000000e+00,1992.0
3,sp|P06621|CBPG_PSES6 Carboxypeptidase G2 OS=Ps...,gi|1480890214|ref|WP_119552493.1| M20/M25/M40 ...,415,MRPSIHRTAIAAVLATAFVAGTALAQKRDNVLFQAATDEQPAVIKT...,MRPSIHRTA+AA+LATAF+A A AQKRDNVLFQAATDEQPAVIKT...,MRPSIHRTALAALLATAFLAPAAWAQKRDNVLFQAATDEQPAVIKT...,1,415,1,415,389,405,0,0.000000e+00,1978.0
4,sp|P06621|CBPG_PSES6 Carboxypeptidase G2 OS=Ps...,gi|1011471139|ref|WP_062367304.1| M20/M25/M40 ...,415,MRPSIHRT-AIAAVLATAFVAGTALAQKRDNVLFQAATDEQPAVIK...,MRP RT A + AQKRDNVLFQAATD QPAV+K...,MRPFNQRTMLAALLATALLAPAAGWAQKRDNVLFQAATDAQPAVLK...,1,415,1,416,374,393,1,0.000000e+00,1938.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4996,sp|P06621|CBPG_PSES6 Carboxypeptidase G2 OS=Ps...,gi|1510926788|ref|WP_122926114.1| ArgE/DapE fa...,415,EQPAVIKTLEKLVNIETGT----GDAEGIAAAGNFLEAELKNLGFT...,E+ +I ++ L+ I++ D G +L+A+L +G ...,ERDELIGLVQDLIRIDSVNPYLDADGPGEREMAAYLQAKLVEMGLE...,39,261,6,223,70,117,19,2.135720e-12,190.0
4997,sp|P06621|CBPG_PSES6 Carboxypeptidase G2 OS=Ps...,gi|1460625799|ref|WP_116468663.1| hydrolase [S...,415,LVVGDNIVGKIKGRGGKNLLLMSHMDTVY-LKGILAKAPFRVEGDK...,L G+++ ++ +L HMDTV+ + +R +G ...,LAHGEHLHLVVRPEAPIQMLFTGHMDTVFGADHVFQHGQWRTDG-T...,89,411,78,396,93,150,18,2.144680e-12,190.0
4998,sp|P06621|CBPG_PSES6 Carboxypeptidase G2 OS=Ps...,gi|1718634206|ref|WP_145030192.1| M20/M25/M40 ...,415,LFQAATDEQPAVIKTLEKLVNIETGTGDAEGIAAAGNFLEAELKNL...,+ +A T E A I + +L++I +G+ +G+A +FL +LK ...,MARAKTSENKAAIDLVLELLSISGKSGEEKGVA---DFLVKKLKGA...,32,237,1,210,73,106,24,2.197150e-12,190.0
4999,sp|P06621|CBPG_PSES6 Carboxypeptidase G2 OS=Ps...,gi|1393435437|ref|WP_109798908.1| M20/M25/M40 ...,415,IAAVLATAFVAGTALAQKRDN----VLFQAATDEQPAVIKTLEKLV...,IA+ LA + A A+A+ ++ + EQ +K LE LV...,IASALALSCSAVQAMAKGPESGPEARMIATIDAEQTRTLKFLETLV...,10,339,7,361,96,151,27,2.322320e-12,190.0


In [10]:
# Extract the sequence of subject proteins (hits) aligned with the query to build the MSA

input_path = Path(blast_input)
msa = []

with open("{}/{}_blast_msa.fasta".format(input_path.parent, input_path.stem), "w") as fout:
    for index, row in df.iterrows():
        mapped_seq = ["-"] * blast_record.query_length  # Empty list of length = query_length
        c = 0
        if row["eval"] < 0.001:
        #     print(row)
            for l_q, l_s in zip(row['query_seq'], row['subject_seq']):
                if l_q != " " and l_q != '-':
                    mapped_seq[row["query_start"] + c -1] = l_s if l_s != " " else "-"
                    c += 1
            fout.write(">{}\n{}\n".format(row["subject_id"], "".join(mapped_seq)))



# Generate a better MSA

Generate a new, possibly better, MSA starting from the full sequences of hits retrieved by Blast using the Clustal Omega method

1. Extract the protein IDs from previous files

```
    # SwissProt IDs
    awk -F"|" '/>/ {print $2}' data/P06621_sprot_blast_msa.fasta > tmp
    
    # RefSeq IDs (ex. WP_094475116.1)
    awk -F"|" '/>/ {print $4}' data/P06621_refseq_selected_blast_msa.fasta > tmp2
   
``` 

2. Download the sequences from UniProt using the **Retrieve/ID mapping** web tool
    * For SwissProt IDs select "From --> UniProtKB, To UniProtKB. Download into **P06621_sprot_blast_mapped.fasta** 
    * For RefSeq selected "From --> RefSeq Protein, To UniProtKB. Select "View UniParc results" to increase the number of retrieved sequences. Download into **P06621_refseq_selected_blast_mapped.fasta**


3. Use [ClustalO](https://www.ebi.ac.uk/Tools/msa/clustalo/) to generate new MSAs. Select the *Fasta* output format and save into
    * **P06621_sprot_blast_clustal.fasta**
    * **P06621_refseq_selected_blast_clustal.fasta**


4. Compare the Blast MSA and the ClustalO MSA in JalView

```
    java -jar jalview-all-2.11.1.4-j1.8.jar
```

# Build PSSMs

To build PSSM from a MSA file it is necesary to use the command line version of Blast

#### Install Blast (~250 Mb)
```
wget ftp://ftp.ncbi.nlm.nih.gov/blast/executables/blast+/LATEST/ncbi-blast-2.12.0+-x64-linux.tar.gz
tar -xzf ncbi-blast-2.12.0+-x64-linux.tar.gz

```

#### Create a PSSM from a Fasta MSA 

Before running the command remove *gi|1774282670|ref|WP_153585536.1|* from the **data/P06621_refseq_selected_blast_msa.fasta** file, for some reasons it breaks the PSSM build

The content of the file in the -subject option is irrelevant, just use a valid fasta file
```
ncbi-blast-2.12.0+/bin/psiblast -subject data/P06621.fasta -in_msa data/P06621_sprot_blast_msa.fasta -out_pssm data/P06621_sprot_blast_msa.pssm -out_ascii_pssm data/P06621_sprot_blast_msa.pssm_ascii

ncbi-blast-2.12.0+/bin/psiblast -subject data/P06621.fasta -in_msa data/P06621_refseq_selected_blast_msa.fasta -out_pssm data/P06621_refseq_selected_blast_msa.pssm -out_ascii_pssm data/P06621_refseq_selected_blast_msa.pssm_ascii
```

# Build HMM

Install HMMER (~18 Mb)
```
wget http://eddylab.org/software/hmmer/hmmer.tar.gz
tar -xzf hmmer.tar.gz
./configure
make
```

Generate HMMs from MSAs
```
hmmer-3.3.2/src/hmmbuild data/P06621_sprot_blast_msa.hmm data/P06621_sprot_blast_msa.fasta

hmmer-3.3.2/src/hmmbuild data/P06621_refseq_selected_blast_msa.hmm data/P06621_refseq_selected_blast_msa.fasta
```

# Build HMM and PSSM from ClustalO MSAs

Generate PSSMs
```
ncbi-blast-2.12.0+/bin/psiblast -subject data/P06621.fasta -in_msa data/P06621_sprot_blast_clustal.fasta -out_pssm data/P06621_sprot_blast_clustal.pssm -out_ascii_pssm data/P06621_sprot_blast_clustal.pssm_ascii

ncbi-blast-2.12.0+/bin/psiblast -subject data/P06621.fasta -in_msa data/P06621_refseq_selected_blast_clustal.fasta -out_pssm data/P06621_refseq_selected_blast_clustal.pssm -out_ascii_pssm data/P06621_refseq_selected_blast_clustal.pssm_ascii
```

Generate HMMs
```
hmmer-3.3.2/src/hmmbuild data/P06621_sprot_blast_clustal.hmm data/P06621_sprot_blast_clustal.fasta

hmmer-3.3.2/src/hmmbuild data/P06621_refseq_selected_blast_clustal.hmm data/P06621_refseq_selected_blast_clustal.fasta
```

# Evaluation

Compare performance of your models in comparison with Pfam **Peptidase_M20.hmm**. 

1. Evaluate the number of hits retrieved by your models and those retrived by the Pfam domain aginst the SwissProt database.

    * **PSSMs** You can use the web service. When you select Psi-Blast it is possible to provide a PSSM file as input

    * **HMMs** You can use HMMSEARCH from HMMER web site


2. Evaluate the overlap between the hits retrieved by your models and the hits retrived by the Pfam model

    * You need to download sequence IDs of the matched sequences 


3. Evaluate False Positive matches (if any) and decide if they are good family candidates missed by Pfam but not by your model

    * You need to look at alternative information to judge. Statistical parameters, curated annotations, etc...
    


# Improve your model

Manually edit the MSA in order to achieve better precision and/or recall